# Importing CSV Dataset

In [298]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit
import math
import numpy as np
import chart_studio.plotly as py
from plotly.graph_objs import *

In [305]:
df = pd.read_csv('volcanic_activity.csv',sep =';', decimal = ',')
df.describe()

,Year,Month,Day,Latitude,Longitude,Elevation,Volcano Explosivity Index (VEI),DEATHS,DEATHS_DESCRIPTION,MISSING,...,TOTAL_DEATHS,TOTAL_DEATHS_DESCRIPTION,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION
count,658.000000,552.000000,499.000000,658.000000,658.000000,658.000000,570.000000,310.000000,425.000000,5.000000,...,322.000000,438.000000,4.000000,8.000000,64.000000,87.000000,10.000000,184.000000,27.000000,98.000000
mean,1671.405775,6.331522,15.244489,16.306745,49.928179,1936.869301,2.887719,609.219355,1.656471,391.400000,...,1136.875776,1.714612,489.000000,2.125000,240.734375,1.666667,266.530000,1.760870,1170.037037,2.234694
std,786.005793,3.321910,9.065156,26.102522,100.351199,1244.770279,1.338108,2889.037021,1.013872,763.961583,...,7417.486636,1.042930,845.386302,1.125992,1252.329312,0.897887,636.873961,0.956651,2265.969846,0.961110
min,-4360.000000,1.000000,1.000000,-62.970000,-178.470000,-642.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
25%,1730.000000,3.750000,7.500000,-6.102000,-19.700000,1053.000000,2.000000,2.000000,1.000000,3.000000,...,2.000000,1.000000,59.250000,1.000000,3.750000,1.000000,3.475000,1.000000,32.000000,1.000000
50%,1895.000000,7.000000,15.000000,13.736000,110.442000,1695.000000,3.000000,7.000000,1.000000,78.000000,...,9.000000,1.000000,99.000000,2.000000,13.000000,1.000000,9.750000,1.000000,90.000000,2.000000
75%,1967.000000,9.000000,23.000000,36.404000,129.967750,2565.000000,4.000000,80.000000,2.000000,120.000000,...,117.750000,3.000000,528.750000,3.000000,66.250000,2.000000,18.000000,3.000000,835.500000,3.000000
max,2014.000000,12.000000,31.000000,65.730000,177.180000,5967.000000,7.000000,30000.000000,4.000000,1755.000000,...,117000.000000,4.000000,1755.000000,4.000000,10000.000000,4.000000,2000.000000,4.000000,9000.000000,4.000000


In [300]:
df = df[df['Year'] >= 1900]
df = df[(df['Agent'].str.contains('T',na=False))]
df = df.drop(columns=['Associated Tsunami?', 'Associated Earthquake?',
       'Location', 'Country', 'Elevation',
       'Type', 'Status', 'Time','Agent',
       'DEATHS', 'DEATHS_DESCRIPTION', 'MISSING', 'MISSING_DESCRIPTION',
       'INJURIES', 'INJURIES_DESCRIPTION', 'DAMAGE_MILLIONS_DOLLARS',
       'DAMAGE_DESCRIPTION', 'HOUSES_DESTROYED',
       'HOUSES_DESTROYED_DESCRIPTION', 'TOTAL_DEATHS',
       'TOTAL_DEATHS_DESCRIPTION', 'TOTAL_MISSING',
       'TOTAL_MISSING_DESCRIPTION', 'TOTAL_INJURIES',
       'TOTAL_INJURIES_DESCRIPTION', 'TOTAL_DAMAGE_MILLIONS_DOLLARS',
       'TOTAL_DAMAGE_DESCRIPTION', 'TOTAL_HOUSES_DESTROYED',
       'TOTAL_HOUSES_DESTROYED_DESCRIPTION'])
df = df.dropna()
df = df.reset_index()
df['Volume of erupted Tephra (km^3)'] = [0.0001*10**i for i in df['Volcano Explosivity Index (VEI)'].values]
df.head()

,index,Year,Month,Day,Name,Latitude,Longitude,Volcano Explosivity Index (VEI),Volume of erupted Tephra (km^3)
0,338,1900,7.0,17.0,Adatara,37.620,140.280,2.0,0.010
1,348,1902,10.0,24.0,Santa Maria,14.756,-91.552,6.0,100.000
2,350,1903,8.0,30.0,Okataina,-38.120,176.500,1.0,0.001
3,352,1905,3.0,10.0,Vesuvius,40.821,14.426,2.0,0.010
4,354,1906,4.0,4.0,Vesuvius,40.821,14.426,3.0,0.100


# Visualisation

In [301]:
fig = px.scatter_geo(df,
                    lat='Latitude',
                    lon='Longitude',
                    size = 'Volcano Explosivity Index (VEI)',
                    color = 'Volcano Explosivity Index (VEI)',
                    hover_name = 'Name',
                    labels={"Volcano Explosivity Index (VEI)": "VEI"},
                    projection ='natural earth',
                    title = "Volcanic activity from 1900 to 2010")
fig.show()

In [302]:
vol_year = df.groupby(['Year'])['Volcano Explosivity Index (VEI)'].sum()


fig = px.bar(vol_year)
fig.update_layout(showlegend = False,
                 xaxis=dict(title = "Year",color = 'black'),
                 yaxis=dict(showgrid = True,
                            title = "Sum of VEI over each year",
                            color = 'black'),
                 title = "Volcanic activity over years from 1900 to 2010")
fig.show()

In [303]:
Tephra_year = df.groupby(['Year'])['Volume of erupted Tephra (km^3)'].sum()


fig = px.bar(Tephra_year)
fig.update_layout(showlegend = False,
                 xaxis=dict(title = "Year",color = 'black'),
                 yaxis=dict(showgrid = True,
                            title = "Volume of erupted Tephra (km^3)",
                            color = 'black'),
                 title = "Volcanic activity over years from 1900 to 2010")
fig.show()